In [48]:
from __future__ import print_function
import torch
from torch.autograd import Variable, Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe, CharNGram

import scipy.io as sio
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from minimax_entropy import MinimaxEntropyEstimator

# https://github.com/pytorch/text
# http://cogcomp.org/Data/QA/QC/

In [49]:
entro = MinimaxEntropyEstimator('poly_coeff_entro.mat', gpu=True)

In [65]:
batch_size = 64
dim = 300

In [66]:
TEXT.build_vocab(train, vectors=[GloVe(name='42B', dim=str(dim)), CharNGram()])
LABEL.build_vocab(train)

train_iter, test_iter = datasets.TREC.iters(batch_size=batch_size)


In [63]:
# let's start off padding the rest to be zero's to get to 300 dim
def list_batches(t_iter, cap=100):
    batches_text, batches_label = [], []
    ct = 0
    batch = next(iter(train_iter))
    while batch and ct < 100:
        batches_text.append(batch.text)
        batches_label.append(batch.label)
        batch = next(iter(train_iter))
        ct += 1
#         print (ct, batch.text.data.size())
    return batches_text, batches_label

train_text, train_label = list_batches(train_iter)
test_text, test_label = list_batches(test_iter)

In [ ]:
# x = np.zeros((dim, 1))
# x[:test_text[:,1].size()] = test_text[:,0]

In [64]:
def forward(self, inp, hidden, lengths):
    embeddings = self.encoder(inp)
    packed = pack_padded_sequence(embeddings, lengths, batch_first=True)
    output, hidden = self.rnn(packed, hidden)
    output, _ = pad_packed_sequence(output, batch_first=True)

Variable containing:

Columns 0 to 12 
   36     4     4    14     4     4     4    48  1707     4     4     4     4
   21    37    85   717     5    15    15  1737     5    11    10   563  8086
 2909    20     6   197    19   159  4315   738     8     3     3     6   173
  839  1262  1066   947     3   220    17   129  3345  1116  1118   195  6187
  361  1229     5     3  9343  7385  6058  3158     6     6   209  8325     3
    7   118     8  2096  9320   566    89   131    38  5143     6    15   143
 2465    32  6820  3500    18  1152    16   648    37  5604  3843    50  8882
    2     2     2     2     2     2     2     2     2     2     2     2     2

Columns 13 to 25 
    4     4     4     9    14     9    25    14    36     9     4     4     4
  108     5    20    22     5    23     5    11    21    23  7438     5    37
   56  2885     3    30     3    54     3     3     3  1047   120     3    45
 1651  6794  6637    55    19    21   297    28    61    15    22  1683     3
 1258 

In [59]:
# for i in range(len(test_label)):
#     print (np.unique(test_label[i].data.cpu().numpy()))

In [40]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = batch_size, dim, 100, 6

print (test_text[0].size(), test_label[0].size())
# # Create random Tensors to hold inputs and outputs, and wrap them in Variables.
# x = Variable(torch.randn(N, D_in))
# y = Variable(torch.randn(N, D_out), requires_grad=False)

# # Use the nn package to define our model as a sequence of layers. nn.Sequential
# # is a Module which contains other Modules, and applies them in sequence to
# # produce its output. Each Linear Module computes output from input using a
# # linear function, and holds internal Variables for its weight and bias.
# model = torch.nn.Sequential(
#           torch.nn.Linear(D_in, H),
#           torch.nn.ReLU(),
#           torch.nn.Linear(H, D_out),
#         )


torch.Size([11, 64]) torch.Size([64])


In [4]:
class Net(nn.Module):
    def __init__(self, dropout=0.):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(dropout)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        self._dropout = dropout

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.dropout(F.relu(self.fc1(x)), p=self._dropout, training=self.training)
        x = F.softmax(self.fc2(x))
        return x

In [5]:
model = Net(dropout=dropout)
model.cuda()

optimizer = optim.Adam(model.parameters())

def train(epoch):
    model.train()
    
    target_onehot = Variable(torch.DoubleTensor(batch_size, 10)).cuda()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        target_onehot.data.zero_()
        for i in range(batch_size):
            target_onehot[i, target.data[i]] = 1.
        
        output = model(data).double()
        
        loss = Variable(torch.zeros(1)).double().cuda()
        for i in range(batch_size):
            loss += entro.cross_entro_loss(output[i], target_onehot[i])
        loss.backward()
        
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        
        if batch_idx == 100:
            break
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 207.984918
Train Epoch: 1 [640/60000 (1%)]	Loss: 199.785011
Train Epoch: 1 [1280/60000 (2%)]	Loss: 174.420898
Train Epoch: 1 [1920/60000 (3%)]	Loss: 134.108899
Train Epoch: 1 [2560/60000 (4%)]	Loss: 79.123192
Train Epoch: 1 [3200/60000 (5%)]	Loss: 61.239456
Train Epoch: 1 [3840/60000 (6%)]	Loss: 56.735480
Train Epoch: 1 [4480/60000 (7%)]	Loss: 73.298097
Train Epoch: 1 [5120/60000 (9%)]	Loss: 61.001399
Train Epoch: 1 [5760/60000 (10%)]	Loss: 68.880989
Train Epoch: 1 [6400/60000 (11%)]	Loss: 40.949460

Test set: Average loss: -0.8354, Accuracy: 9068/10000 (91%)

